In [25]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src/')
import dataset_prep

In [99]:
selected_columns = ["time_diff",
"nb_packets_sent",
"max_speed1",
"max_speed2",
"hed_noise_x",
"sendTime",
"rcvTime",
"hed_noise_y",
"receiver",
"sender",
"label"
]

In [100]:
df = dataset_prep.import_dataset('../data/DoS_0709_new_columns.csv', selected_columns)

In [101]:
df["sender"].value_counts()

17853    10919
18315    10793
22371    10033
18213     9710
17733     9605
         ...  
7911        10
7011        10
21399        8
159          7
10785        6
Name: sender, Length: 4067, dtype: int64

In [102]:
df["receiver"].value_counts()

18357    6225
18003    6097
18063    5097
18669    5086
17841    5085
         ... 
22029      11
19629       9
7911        9
24411       8
7011        5
Name: receiver, Length: 4067, dtype: int64

In [103]:
dataset_prep.clean_dataset(df)

Nombre de lignes avant nettoyage :  4753433
Nombre de lignes après nettoyage :  4605219


In [104]:
senders = np.unique(df["sender"].values)
senders

array([    9,    15,    21, ..., 24399, 24405, 24411], dtype=int64)

In [105]:
print(len(senders))

4064


In [107]:
df.loc[df['sender'] == senders[0]]

,rcvTime,sendTime,sender,receiver,hed_noise_x,hed_noise_y,label,max_speed1,max_speed2,nb_packets_sent,time_diff
1,25280.602880,25280.602880,9,21,18.301358,9.010233,0,5.28049,14.501931,96,0.999998
2,25281.602878,25281.602878,9,21,18.420074,5.279148,0,5.28049,14.501931,96,0.999998
3,25282.602871,25282.602871,9,21,10.612548,3.140676,0,5.28049,14.501931,96,0.999994
5,25213.602877,25213.602877,9,45,11.881607,2.219595,0,5.28049,14.501931,96,1.000001
6,25214.602878,25214.602878,9,45,11.618620,2.209780,0,5.28049,14.501931,96,1.000001
...,...,...,...,...,...,...,...,...,...,...,...
90,25291.602880,25291.602880,9,219,9.762936,4.073993,0,5.28049,14.501931,96,1.000002
91,25292.602869,25292.602869,9,219,9.443977,6.379100,0,5.28049,14.501931,96,0.999989
93,25290.602878,25290.602878,9,225,10.089412,2.613200,0,5.28049,14.501931,96,1.000002
94,25291.602880,25291.602880,9,225,9.762936,4.073993,0,5.28049,14.501931,96,1.000002


In [110]:
sequence_arr = df.loc[df['sender'] == senders[0]][0:10]
sequence = sequence_arr.drop(["label"], axis=1)

sequence

,rcvTime,sendTime,sender,receiver,hed_noise_x,hed_noise_y,max_speed1,max_speed2,nb_packets_sent,time_diff
1,25280.602880,25280.602880,9,21,18.301358,9.010233,5.28049,14.501931,96,0.999998
2,25281.602878,25281.602878,9,21,18.420074,5.279148,5.28049,14.501931,96,0.999998
3,25282.602871,25282.602871,9,21,10.612548,3.140676,5.28049,14.501931,96,0.999994
5,25213.602877,25213.602877,9,45,11.881607,2.219595,5.28049,14.501931,96,1.000001
6,25214.602878,25214.602878,9,45,11.618620,2.209780,5.28049,14.501931,96,1.000001
7,25215.602879,25215.602879,9,45,11.630224,2.196400,5.28049,14.501931,96,1.000001
8,25216.602880,25216.602880,9,45,11.316523,2.186352,5.28049,14.501931,96,1.000001
9,25217.602881,25217.602881,9,45,11.349668,2.188293,5.28049,14.501931,96,1.000001
10,25218.602882,25218.602882,9,45,11.358689,2.183506,5.28049,14.501931,96,1.000001
11,25219.602870,25219.602870,9,45,11.403368,2.191527,5.28049,14.501931,96,0.999988


Amélioration à faire :

Séléctionner seulement les sender avec value_counts > 20

In [112]:
sequence_df = pd.DataFrame()
sequence_array = []
label_array = []

for sender in senders:
    length = df.loc[df['receiver'] == sender].shape[0]
    slide = 10
    start = 0
    end = 20
    # On vérifie qu'il est possible de faire une séquence de taille 20
    while length > 20:
        # Extraction par tranche de 20 avec une inter de 10
        sequence = df.loc[df['sender'] == sender][start:end]

        # Labels correspondant
        labels = sequence["label"]

        # Autres colonnes
        sequence = sequence.drop(["label"], axis=1)

        # Attribution des tableaux numpy
        sequence_array.append(np.asarray(sequence))
        label_array.append(labels)

        # Mise à jour des variables
        start += slide
        end += slide
        length -= 20